In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from collections import Counter
import os
import sys
import re
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix

import keras
from keras.layers import Dense, ReLU, Input, GlobalAveragePooling2D
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras.applications.vgg16 import VGG16
from keras.models import Model

In [ ]:
tqdm.pandas()
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 150)
from matplotlib import rcParams
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = ['Hiragino Maru Gothic Pro', 'Yu Gothic', 'Meirio', 'Takao', 'IPAexGothic', 'IPAPGothic', 'VL PGothic', 'Noto Sans CJK JP']

In [ ]:
filePath = "data/tinder.xlsx"
imagePath = "data/photos"

In [ ]:
df = pd.read_excel(filePath)
df.drop_duplicates(inplace=True, subset="id")
df.set_index("id", inplace=True)
df["bio"] = df["bio"].fillna("")

In [ ]:
X=[]
y=[]
for fileName in tqdm(os.listdir(imagePath)):
    try:
        id_ = re.match("([a-z0-9]*)-\d( \(\d\))?.jpg",fileName).group(1)
        match = df.loc[id_]["match"]
        filePath = os.path.join(imagePath, fileName)
        img = cv2.imread(filePath)
        img = cv2.resize(img, (120,120))
        X.append(img)
        y.append(match)
    except:
        pass
X=np.asarray(X)
y=np.asarray(y)

In [ ]:
X = (X/255).astype(np.float16)

In [ ]:
np.save("X.npy", X)
np.save("y.npy", y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=8888)

In [ ]:
def getModel():
    input_tensor = Input(shape=(120,120,3))
    model = VGG16(weights="imagenet", include_top=False)
    x = model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024)(x)
    x = ReLU()(x)
    predictions = Dense(2, activation="softmax")(x)
    model = Model(inputs=model.input, outputs=predictions)
    for layer in model.layers[:18]:
        layer.trainable=False
    return model

In [ ]:
model = getModel()

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=["accuracy"])

In [ ]:
model.fit(X_train, to_categorical(y_train), epochs=30, validation_data=(X_test, to_categorical(y_test)))

In [ ]:
y_pred=model.predict(X_test)

In [ ]:
y_pred=np.exp(y_pred)
y_pred=(y_pred/np.sum(y_pred,axis=1).reshape(-1,1))[:,1]

In [ ]:
roc_auc_score(y_test,y_pred)